In [ ]:
!unzip Dataset.zip
!ls

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
#Declare the vectorizers
count_vectorizer=CountVectorizer(stop_words='english',max_features=100)
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_features=100)
hashing_vectorizer=HashingVectorizer(stop_words='english')

In [ ]:
label_list=[]
text_list=[]
file_list=[]
fileDir="Section1-Chapter3"
df=pd.read_csv(fileDir+"/Labels.csv")

for row in df.itertuples(index=True, name='Pandas'):
    fileName=row.FileName+'.asm'
    with open("Section1-Chapter3/"+fileName, encoding='utf-8', errors='ignore') as f:
        content=f.read()
    content.replace(",", " ")
    content.replace('"', " ")
    text_list.append(content)
    file_list.append(fileName)
    label_list.append(row.Label)           

In [ ]:
#Create dataframe for Count Vectorizer
dtm=count_vectorizer.fit_transform(text_list)
df_count=pd.DataFrame(dtm.toarray(),columns=count_vectorizer.get_feature_names())
df_count['FileName']=pd.Series(file_list)
df_count['Label']=pd.Series(label_list)


In [ ]:
#Create dataframe for TF-IDF Vectorizer
dtm=tfidf_vectorizer.fit_transform(text_list)
df_tfidf=pd.DataFrame(dtm.toarray(),columns=tfidf_vectorizer.get_feature_names())
df_tfidf['FileName']=pd.Series(file_list)
df_tfidf['Label']=pd.Series(label_list)

In [ ]:
df_count.head()

In [ ]:
df_tfidf.head()

In [ ]:
#Feature Engneering section
df_count.to_csv('count_vectorizer.csv.zip', index=False,compression="zip") 
df_tfidf.to_csv('tfidf_vectorizer.csv.zip', index=False,compression="zip") 

In [ ]:
df_count=pd.read_csv("count_vectorizer.csv.zip")

In [ ]:
df_tfidf=pd.read_csv("tfidf_vectorizer.csv.zip")

In [ ]:
df_count.head()

In [ ]:
#Select the Feature and Target lable matrix 
Label_CV=df_count['Label']
Features_CV = df_count.drop( columns=['FileName', 'Label'], axis=1)

In [ ]:
Label_TF=df_tfidf['Label']
Features_TF= df_tfidf.drop( columns=['FileName', 'Label'], axis=1)

In [ ]:
#Normalizing the input and splitting train test data 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_CF_Normalized=StandardScaler().fit_transform(Features_CV)
X_TF_Normalized=StandardScaler().fit_transform(Features_TF)

X_CF_Train, X_CF_Test, Y_CF_Train, Y_CF_Test= train_test_split(X_CF_Normalized,Label_CV,test_size=0.20, shuffle=True)

X_TF_Train, X_TF_Test, Y_TF_Train, Y_TF_Test= train_test_split(X_TF_Normalized,Label_TF,test_size=0.20, shuffle=True)

In [ ]:
#Train the dataset
from sklearn.linear_model import LogisticRegression
logReg_cf=LogisticRegression()
logReg_cf.fit(X_CF_Train,Y_CF_Train)


logReg_tf=LogisticRegression()
logReg_tf.fit(X_TF_Train,Y_TF_Train)

In [ ]:
#Run the above model and get predictions 
predictions_cf=logReg_cf.predict(X_CF_Test)

predictions_tf=logReg_tf.predict(X_TF_Test)


In [ ]:
#Print Util Function
from sklearn.metrics import accuracy_score
from prettytable import PrettyTable
def print_stats_metrices(algorithm_name, y_test, y_pred):
    x = PrettyTable()
    x.field_names = ["Algorithm", "Accuracy"]
    
    acc_score=accuracy_score(y_test, y_pred)
    x.add_row([algorithm_name,acc_score]);
    print(x)

In [ ]:
print_stats_metrices('LogisticRegression',Y_CF_Test,predictions_cf)

In [ ]:
print_stats_metrices('LogisticRegression',Y_TF_Test,predictions_tf)